In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

base_path = '/content/drive/MyDrive/x_ray_chest/results/yolo'
ver = 'v1'

# 新しいフォルダを生成
weight_folder = 'weight'
train_folder = 'train'
test_folder = 'test'
test_img_folder = 'images'

weight_folder_path = os.path.join(base_path, ver, weight_folder)
train_folder_path = os.path.join(base_path, ver, train_folder)
test_folder_path = os.path.join(base_path, ver, test_folder)
test_img_path = os.path.join(base_path, ver, test_folder, test_img_folder)

In [ ]:
yolo_dataset_path = '/content/drive/MyDrive/x_ray_chest/results/yolo/dataset'
ver = 'v1'

base_path = os.path.join(yolo_dataset_path, ver)

dataset_images_path = os.path.join(base_path, 'images')
dataset_labels_path = os.path.join(base_path, 'labels')

images_train_path = os.path.join(dataset_images_path, 'train')
images_val_path = os.path.join(dataset_images_path, 'val')
images_test_path = os.path.join(dataset_images_path, 'test')

labels_train_path = os.path.join(dataset_labels_path, 'train')
labels_val_path = os.path.join(dataset_labels_path, 'val')
labels_test_path = os.path.join(dataset_labels_path, 'test')

mkdir_list = [base_path, dataset_images_path, dataset_labels_path, images_train_path, images_val_path, images_test_path, labels_train_path, labels_val_path, labels_test_path]

for mkdir_path in mkdir_list:
    os.mkdir(mkdir_path)


In [ ]:
from glob import glob
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
BBox_List_path = '/content/drive/MyDrive/x_ray_chest/BBox_List_2017.csv'
Data_Entry_path = '/content/drive/MyDrive/x_ray_chest/Data_Entry_2017_v2020.csv'

BBox_List_df = pd.read_csv(BBox_List_path)
Data_Entry_df = pd.read_csv(Data_Entry_path)

In [ ]:
data_df = BBox_List_df.merge(Data_Entry_df, on='Image Index', how='inner')
data_df = data_df.set_axis(['img_name', 'label', 'bbox_x', 'bbox_y', 'bbox_w', 'bbox_h', 'nan1', 'nan2', 'nan3', 'labels', 'follow_up', 'id', 'age', 'gender', 'view_posi', 'ori_img_w', 'ori_img_h', 'pixel_mm_x', 'pixel_mm_y'], axis=1)
data_df = data_df.drop(['nan1', 'nan2', 'nan3', 'labels', 'follow_up', 'id', 'age', 'gender', 'view_posi', 'pixel_mm_x', 'pixel_mm_y', 'ori_img_w', 'ori_img_h'], axis=1)

In [ ]:
import torch
from torch.utils.data import Dataset
import torchvision
from torchvision import datasets
from torchvision import transforms
import h5py

In [ ]:
# 画像データをimagesに格納

file_path = '/content/drive/MyDrive/x_ray_chest/file.hdf5'

images = []

with h5py.File(file_path, "r") as f:
    group = f["/images"]

    for i in range(len(data_df['img_name'].unique())):
        file_name = data_df['img_name'].unique()[i]

        if file_name in group:
            dataset = group[file_name]
            img_data = np.array(dataset)
            img_tensor = transforms.Compose([transforms.ToTensor()])(img_data)
            images.append(img_tensor)

In [ ]:
count = 0

for img_tensor in images:
    if img_tensor.shape == torch.Size([3, 1024, 1024]):
        count += 1

print(count)

880


In [ ]:
data_df['center_x_per'] = (data_df['bbox_x'] + data_df['bbox_w'] / 2) / 1024
data_df['center_y_per'] = (data_df['bbox_y'] + data_df['bbox_h'] / 2) / 1024
data_df['width_per'] = data_df['bbox_w'] / 1024
data_df['height_per'] = data_df['bbox_h'] / 1024

data_df = data_df.drop(['bbox_x', 'bbox_y', 'bbox_w', 'bbox_h'], axis=1)

data_df.head()

,img_name,label,center_x_per,center_y_per,width_per,height_per
0,00013118_008.png,Atelectasis,0.262182,0.572864,0.084746,0.077331
1,00014716_007.png,Atelectasis,0.760593,0.281532,0.181144,0.306144
2,00029817_009.png,Atelectasis,0.292373,0.415554,0.151483,0.211864
3,00014687_001.png,Atelectasis,0.778072,0.510364,0.137712,0.054025
4,00017877_001.png,Atelectasis,0.742585,0.594562,0.195975,0.076271


In [ ]:
import pandas as pd

replace_dict = {'Atelectasis':'0', 'Cardiomegaly':'1', 'Effusion':'2', 'Infiltrate':'3', 'Mass':'4', 'Nodule':'5', 'Pneumonia':'6', 'Pneumothorax':'7'}

data_df['label'] = data_df['label'].replace(replace_dict)

print(data_df)


             img_name label  center_x_per  center_y_per  width_per  height_per
0    00013118_008.png     0      0.262182      0.572864   0.084746    0.077331
1    00014716_007.png     0      0.760593      0.281532   0.181144    0.306144
2    00029817_009.png     0      0.292373      0.415554   0.151483    0.211864
3    00014687_001.png     0      0.778072      0.510364   0.137712    0.054025
4    00017877_001.png     0      0.742585      0.594562   0.195975    0.076271
..                ...   ...           ...           ...        ...         ...
979  00029464_003.png     0      0.337611      0.428519   0.200000    0.125556
980  00001320_003.png     0      0.238722      0.546852   0.071111    0.028889
981  00013187_002.png     0      0.554833      0.640185   0.721111    0.171111
982  00025769_001.png     0      0.735944      0.590185   0.101111    0.062222
983  00026920_000.png     0      0.394278      0.461852   0.117778    0.052222

[984 rows x 6 columns]


In [ ]:
dataset = []
for i in range(len(images)):
    data = (data_df['img_name'].unique()[i])
    dataset.append(data)

In [ ]:
train, val, test = torch.utils.data.random_split(dataset=dataset, lengths=[736, 72, 72], generator=torch.Generator().manual_seed(42))

train_loader = torch.utils.data.DataLoader(train, batch_size = 1, shuffle=True)
val_loader = torch.utils.data.DataLoader(val, batch_size = 1, shuffle=False)
test_loader = torch.utils.data.DataLoader(test, batch_size = 1, shuffle=False)

In [ ]:
train_list = []
val_list = []
test_list = []

[train_list.append(data[0]) for data in train_loader]
[val_list.append(data[0]) for data in val_loader]
[test_list.append(data[0]) for data in test_loader]

train_txt_path = os.path.join(train_folder_path, 'train_list.txt')
val_txt_path = os.path.join(train_folder_path, 'val_list.txt')
test_txt_path = os.path.join(train_folder_path, 'test_list.txt')

with open(train_txt_path, 'w') as f:
    for item in train_list:
        f.write(str(item) + '\n')

with open(val_txt_path, 'w') as f:
    for item in val_list:
        f.write(str(item) + '\n')

with open(test_txt_path, 'w') as f:
    for item in test_list:
        f.write(str(item) + '\n')


In [ ]:
import shutil

for file_name in train_list:
    from_path = os.path.join('/content/drive/MyDrive/x_ray_chest/images', file_name)
    log_file = shutil.copy(from_path, images_train_path)
    print(log_file)

for file_name in val_list:
    from_path = os.path.join('/content/drive/MyDrive/x_ray_chest/images', file_name)
    log_file = shutil.copy(from_path, images_val_path)
    print(log_file)

for file_name in test_list:
    from_path = os.path.join('/content/drive/MyDrive/x_ray_chest/images', file_name)
    log_file = shutil.copy(from_path, images_test_path)
    print(log_file)


/content/drive/MyDrive/x_ray_chest/results/yolo/dataset/v1/images/train/00021862_004.png
/content/drive/MyDrive/x_ray_chest/results/yolo/dataset/v1/images/train/00011124_000.png
/content/drive/MyDrive/x_ray_chest/results/yolo/dataset/v1/images/train/00021409_001.png
/content/drive/MyDrive/x_ray_chest/results/yolo/dataset/v1/images/train/00010172_001.png
/content/drive/MyDrive/x_ray_chest/results/yolo/dataset/v1/images/train/00025664_002.png
/content/drive/MyDrive/x_ray_chest/results/yolo/dataset/v1/images/train/00030128_002.png
/content/drive/MyDrive/x_ray_chest/results/yolo/dataset/v1/images/train/00027797_000.png
/content/drive/MyDrive/x_ray_chest/results/yolo/dataset/v1/images/train/00028454_016.png
/content/drive/MyDrive/x_ray_chest/results/yolo/dataset/v1/images/train/00006096_010.png
/content/drive/MyDrive/x_ray_chest/results/yolo/dataset/v1/images/train/00030674_000.png
/content/drive/MyDrive/x_ray_chest/results/yolo/dataset/v1/images/train/00012021_081.png
/content/drive/MyDriv

In [ ]:
for file_name in train_list:
    basename = file_name.split('.')[0]
    txt_name =str(basename) + '.txt'
    rows = data_df[data_df['img_name'] == file_name]

    with open(os.path.join(labels_train_path, txt_name), 'w') as f:
        for i in range(len(rows)):
            row = []
            row.append(rows.iloc[i, 1])
            row.append(rows.iloc[i, 2])
            row.append(rows.iloc[i, 3])
            row.append(rows.iloc[i, 4])
            row.append(rows.iloc[i, 5])
            row_data = ' '.join(map(str, row))
            f.write(row_data + '\n')

for file_name in val_list:
    basename = file_name.split('.')[0]
    txt_name =str(basename) + '.txt'
    rows = data_df[data_df['img_name'] == file_name]

    with open(os.path.join(labels_val_path, txt_name), 'w') as f:
        for i in range(len(rows)):
            row = []
            row.append(rows.iloc[i, 1])
            row.append(rows.iloc[i, 2])
            row.append(rows.iloc[i, 3])
            row.append(rows.iloc[i, 4])
            row.append(rows.iloc[i, 5])
            row_data = ' '.join(map(str, row))
            f.write(row_data + '\n')

for file_name in test_list:
    basename = file_name.split('.')[0]
    txt_name =str(basename) + '.txt'
    rows = data_df[data_df['img_name'] == file_name]

    with open(os.path.join(labels_test_path, txt_name), 'w') as f:
        for i in range(len(rows)):
            row = []
            row.append(rows.iloc[i, 1])
            row.append(rows.iloc[i, 2])
            row.append(rows.iloc[i, 3])
            row.append(rows.iloc[i, 4])
            row.append(rows.iloc[i, 5])
            row_data = ' '.join(map(str, row))
            f.write(row_data + '\n')
